# HNSCC RNA-Seq Data

## Download and Parse TCGA RNA-Seq Data Workflow

### McWeeney Lab, Oregon Health & Science University

** Author: Gabrielle Choonoo (choonoo@ohsu.edu) **

## Introduction

This is the step-by-step workflow for downloading and parsing TCGA RNA-Seq Data to a tab-deliminated file that contains genes as rows and patient IDs as columns. Parses all 3 versions of RNA seq: FPKM, FPKM-UQ, and Htseq counts.

Required Files:
* Manifest File (.txt)
* Meta Data (.json)
* This notebook (HNSCC_parse_RNAseq_Notebook.ipynb): [[Download here]](https://raw.githubusercontent.com/gchoonoo/HNSCC_parse_RNAseq_Notebook/master/HNSCC_parse_RNAseq_Notebook.ipynb)

Required R packages:
- `readr`
- `tidyjson`

Recommended software:
* VirtualBox (This workflow can be ran locally, however it can require up to 40GB of storage space)

**Note: this notebook can also be downloaded as shell or R scripts (only the code blocks seen below will be included): 
* [[Download shell script here]](https://raw.githubusercontent.com/gchoonoo/HNSCC_Clinical_Data_Notebook/master/rnaseq.sh)
* [[Download R script here]](https://raw.githubusercontent.com/gchoonoo/HNSCC_Clinical_Data_Notebook/master/parse_rnaseq_local.r)
* [[Download R script here]](https://raw.githubusercontent.com/gchoonoo/HNSCC_Clinical_Data_Notebook/master/parse_rnaseq_virtual.r)

** All code is available on GitHub: [https://github.com/gchoonoo/HNSCC_parse_RNAseq_Notebook](https://github.com/gchoonoo/HNSCC_parse_RNAseq_Notebook) **

## Download TCGA RNA-Seq data

1) https://gdc-portal.nci.nih.gov/

2) Click data

3) Select cancer type (Head and neck) and Gene Expression Quantification for the data type

4) Add all files to cart and go to cart (1638 files)

5) From the Download pull down menu, select Manifest

6) Click the Metadata button to left of the Download button. 

Now you have the required files needed to query TCGA, which is the manifest file (.txt) and the Metadata file (.json)


## Initialize new machine (Ubuntu 64 bit) on Virtual Box. Allocate at least 8 GB of memory and 40GB of storage. 

Once that the machine is ready, log in.

login username: vagrant
pass: vagrant

## Install GDC Client on Virtual Box

### Install gdc client on virtual box
wget https://gdc.nci.nih.gov/files/public/file/gdc-client_v1.0.1_Ubuntu14.04_x64.zip

unzip gdc-client_v1.0.1_Ubuntu14.04_x64.zip

### Verify that program works
./gdc-client

### Create a local folder to be shared on Virutal Box and put the manifest file, metadata, and parse_rnaseq_virtual.r script in this folder.

### Enable guest additions in Virtual Box

### Add yourself as a user in the group and restart the Virtual Box
sudo adduser yourusername vboxsf

### Add shared folder in the machine settings (auto mount)
### The folder will be saved in /media/sf_sharedfolder, where “sharedfolder” is the name of the shared folder

### In this case the shared folder is:

/Users/choonoo/HNSCC_parse_RNAseq/venv_files

## Use GDC Client to download the manifest

### Specify folder where to download files from the manifest:

mkdir files

### Download files

./gdc-client download --no-annotations -m /media/sf_venv_files/gdc_manifest_20170525_163733.txt -d ~/files/

### Note: If you get a notification that asks to overwrite annotations, just hit enter many times. The annotations file isn’t used in this workflow. Be careful not quit out because your download will stop.

### Put the nested files in a new folder:

mkdir data

find /home/vagrant/files/ -mindepth 2 -type f -exec mv -i '{}' /home/vagrant/data ';'

### Gunzip all files

cd /home/vagrant/data
gunzip *.gz

### Check the length of the folder, should be around 1638

## Map the File IDs to the Patient IDs locally

In [ ]:
## Read in libraries
library(readr)
library(tidyjson)

## Read in manifest file
mani <- readr::read_delim("/Users/choonoo/HNSCC_parse_RNAseq/venv_files/gdc_manifest_20170525_163733.txt", delim = "\t")

## ID represents downloaded folders
newManifest <- mani[c("id", "filename")]

## Read in metadata file
metadataFile <- jsonlite::fromJSON("/Users/choonoo/HNSCC_parse_RNAseq/venv_files/metadata.cart.2017-05-25T16_38_26.225791.json")

metadatas <- tidyjson::read_json("/Users/choonoo/HNSCC_parse_RNAseq/venv_files/metadata.cart.2017-05-25T16_38_26.225791.json")

## Parse the attributes of the .json file 
cases <- names(grep("^TCGA", unlist(attributes(metadatas)$JSON[[1]][[1]]), value = TRUE))[[9]]
type <- grep("file_id", names(unlist(attributes(metadatas)$JSON[[1]][[1]])), value = TRUE)[[2]]
idx <- match(c(cases, type), names(unlist(attributes(metadatas)$JSON[[1]][[1]])))

## Loop through file and extract the IDs matched with the file name
metaList <- attributes(metadatas)[["JSON"]][[1L]]
IDS <- lapply(seq_along(metaList), function(i) {
  bcodeIdx <- match("cases.samples.portions.analytes.aliquots.submitter_id",
                    names(unlist(metaList[i])))
  fileIdx <- match("file_name", names(unlist(metaList[i])))
  unlist(metaList[i])[c(bcodeIdx, fileIdx)]
})

bcodeFileID <- data.frame(do.call(rbind, IDS), stringsAsFactors = FALSE)

# Includes primary tumor sample, metastatic, and solid normal tissue expression
unique(sapply(strsplit(bcodeFileID[,1], "-"),"[",4))

# 501 unique patients
length(unique(paste(sapply(strsplit(bcodeFileID[,1], "-"),"[",2),sapply(strsplit(bcodeFileID[,1], "-"),"[",3),sep="-")))

bcodeFileID_v2 = bcodeFileID

bcodeFileID_v2[,2] <- gsub(".gz","",bcodeFileID_v2[,2])

# Save metadata in shared folder
write.table(file="/Users/choonoo/HNSCC_parse_RNAseq/venv_files/metadata_clean.txt",x=bcodeFileID_v2, sep="\t",quote=F,row.names=F)

## Run the virtual pipeline

Rcript /media/sf_venv_files/parse_rnaseq_virtual.r

## Data should be about 546 columns and over 60,000 rows for each of the 3 RNA seq files

## Below is the code used to parse the data in Virtual Box in parse_rnaseq_virtual.r. 
## This is for reference only. Once the script is sourced in Virtual Box, the resulting 3 RNA-seq files will be saved in the shared folder on your local machine.

In [ ]:
# read in the meta data from the shared folder
read.delim("/media/sf_venv_files/metadata_clean.txt", header=T) -> bcodeFileID_v2

# set directory where data is
dir = "/home/vagrant/data"

# get list of files without parcel (logs)  
files = dir(dir)[-grep("parcel",dir(dir))]

fpkm_dir = files[grep("FPKM.txt",files)]

fpkm_uq_dir = files[grep("FPKM-UQ.txt",files)]

htseq_counts_dir = files[grep("htseq.counts",files)]

############################################################

# read in and parse FPKM

fpkm_data = vector("list", length(fpkm_dir))

for(i in 1:length(fpkm_dir)){
  print(i)
  read.delim(file=paste(dir, fpkm_dir[i], sep="/"), header=F) -> fpkm_data[[i]]
  
  row.names(fpkm_data[[i]]) <- fpkm_data[[i]][,1]
  
  fpkm_data[[i]][ , 2, drop = FALSE] -> fpkm_data[[i]]
  
  names(fpkm_data[[i]]) <- bcodeFileID_v2[bcodeFileID_v2[,"file_name"] %in% fpkm_dir[i],1]
}

head(fpkm_data[[1]])
head(fpkm_data[[2]])

do.call(cbind, fpkm_data) -> fpkm_data_full


############################################################

# read in and parse FPKM UQ

fpkm_uq_data = vector("list", length(fpkm_uq_dir))

for(i in 1:length(fpkm_uq_dir)){
  print(i)
  read.delim(file=paste(dir, fpkm_uq_dir[i], sep="/"), header=F) -> fpkm_uq_data[[i]]
  
  row.names(fpkm_uq_data[[i]]) <- fpkm_uq_data[[i]][,1]
  
  fpkm_uq_data[[i]][ , 2, drop = FALSE] -> fpkm_uq_data[[i]]
  
  names(fpkm_uq_data[[i]]) <- bcodeFileID_v2[bcodeFileID_v2[,"file_name"] %in% fpkm_uq_dir[i],1]
}

head(fpkm_uq_data[[1]])
head(fpkm_uq_data[[2]])

do.call(cbind, fpkm_uq_data) -> fpkm_uq_data_full

############################################################

# read in and parse htseq counts

hitseq_data = vector("list", length(htseq_counts_dir))

for(i in 1:length(htseq_counts_dir)){
  print(i)
  read.delim(file=paste(dir, htseq_counts_dir[i], sep="/"), header=F) -> hitseq_data[[i]]
  
  row.names(hitseq_data[[i]]) <- hitseq_data[[i]][,1]
  
  hitseq_data[[i]][ , 2, drop = FALSE] -> hitseq_data[[i]]
  
  names(hitseq_data[[i]]) <- bcodeFileID_v2[bcodeFileID_v2[,"file_name"] %in% htseq_counts_dir[i],1]
}

head(hitseq_data[[1]])
head(hitseq_data[[2]])

do.call(cbind, hitseq_data) -> hitseq_data_full

############################################################

# save data

save(fpkm_data_full, file="/media/sf_venv_files/fpkm_data_full.rda")

save(fpkm_uq_data_full, file="/media/sf_venv_files/fpkm_uq_data_full.rda")

save(hitseq_data_full, file="/media/sf_venv_files/hitseq_data_full.rda")

write.table("/media/sf_venv_files/fpkm_data_full.txt", x= fpkm_data_full, sep="\t", quote=F)

write.table("/media/sf_venv_files/fpkm_uq_data_full.txt", x= fpkm_uq_data_full, sep="\t", quote=F)

write.table("/media/sf_venv_files/hitseq_data_full.txt", x=hitseq_data_full, sep="\t", quote=F)
